In [1]:
import numpy as np
import json
import requests
import pickle
import pandas as pd

# AWS requirements
import boto3
import sagemaker
from sagemaker import get_execution_role
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

from sagemaker.feature_store.feature_group import FeatureGroup


#Query transunion information

def query_transunion(dict_params: dict):
    
    """ Function will use the Transunion API to provide a json response of it
    
    :param dict_params: Dictionary containing the search headers and content, e.g. {'email': 'romoloinsurance@gmail.com', 'phone': '9543946386'}
    :type dict_params: dict
    
    """
    
    headers={
        'Content-type':'application/json',
        'Accept':'application/json'
    }

    url = 'https://api.healthcare.com/third-party/api/v1/third-parties/transunion'
    json_object = json.dumps(dict_params)
    response = requests.post(url=url, headers=headers, data=json_object)
    response_json = json.loads(response.text)
    
    if 'transaction' in response_json.keys():
        if 'output' in response_json['transaction'].keys():
            return response_json['transaction']['output']
    else: 
        return None


# Getting transunion lead information

def process_tu_lead(json_lead):
    
    """ Function takes the Transunion response and process it to a dictionary
    
    :param json_lead: json lead object
    :type json_lead: json
        
    """
    
    if json_lead is None:
        return {}
    personal_data = json_lead.get('data')
    personal_variables = ['email', 'phone', 'first', 'last']

    personal_output = {}

    if personal_data:
        for personal_variable in personal_variables:
            personal_variable_output = personal_data.get(personal_variable)
            if personal_variable_output:
                personal_variable_output = personal_variable_output.lower()
            personal_output[personal_variable] = personal_variable_output


    demo = json_lead.get('demo')
    demo_variables = [('age', 'years'), ('income', 'dollars'), ('children', 'yes'), ('education', 'years'),
                     ('homeowner', 'yes'), ('homevalue', 'dollars'), ('resident', 'years'),
                     ('occupation', 'first'), ('affiliation', 'conservative'), ('affiliation', 'liberal')]

    demo_output = {}

    if demo:
        for demo_var in demo_variables:
            keys = set(demo.keys())

            var_name = demo_var[0]
            sub_var_name = demo_var[1]
            if var_name in keys:
                demo_output[f'{var_name}_{sub_var_name}'] = demo[var_name].get(sub_var_name)

    demo_output.update(personal_output)

    scores = json_lead.get('score')
    scores_output = {}
    if scores:
        for score in scores:
            score_name = score.get('name')
            score_number = score.get('score')

            scores_output[score_name] = int(score_number)

    demo_output.update(scores_output)
    
    return demo_output
            

In [3]:
lead = query_transunion({'phone': int(3124784892)})
processed_lead = process_tu_lead(lead)

In [4]:
print("lead:", lead)

lead: {'address': {'address': '77 ALLEN ST', 'city': 'BUFFALO', 'state': 'NY', 'zip': '142021319'}, 'data': {'input': 'data', 'first': 'JAWHAR', 'last': 'MOHAMMED', 'address': '77 ALLEN ST', 'city': 'BUFFALO', 'company': 'SERRANO MARIA CONSULTANT', 'dob': '199609', 'phone': '3124784892', 'state': 'NY', 'zip': '142021319'}, 'verify': {'input': 'data', 'first': 'exact match', 'last': 'exact match', 'address': 'exact match', 'company': 'exact match', 'dob': 'exact match', 'phone': 'exact match'}, 'message': [{'input': 'data', 'code': 'AA13', 'message': 'Apartment or highrise address'}, {'input': 'data', 'code': 'AP04', 'message': 'Phone is cellular or mobile'}, {'input': 'data', 'code': 'AA17', 'message': 'Address was DPV confirmed for the primary number only, and secondary number information was missing'}, {'input': 'data', 'code': 'AA23', 'message': 'Address is not receiving delivery'}, {'input': 'data', 'code': 'AA17', 'message': 'Address was DPV confirmed for the primary number only, 

In [5]:
print("processed_lead:", processed_lead)

processed_lead: {'email': None, 'phone': '3124784892', 'first': 'jawhar', 'last': 'mohammed', 'ebureau-score-market-201801220508': 222, 'ebureau-score-market-201805170506': 520}
